# 1. Install and Import Dependencies

In [ ]:
%pip install mediapipe opencv-python pandas scikit-learn

In [386]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import csv
import os
import numpy as np

In [387]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

# 2. Capture Landmarks & Export to CSV
<!-- <img src="https://i.imgur.com/AzKNp7A.png">--> 

In [388]:
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [389]:
with open('Foot.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [480]:
class_name = "Incorrect" #dataset path

In [481]:

# cap = cv2.imread("Stance")

directory = "images/Stance/no"
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
   for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".JPG"):
            # Load the image
            image = cv2.imread(os.path.join(directory, filename))

            # Recolor Feed
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False        

            # Make Detections
            results = holistic.process(image)
            # print(results.face_landmarks)

            # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks

            # Recolor image back to BGR for rendering
            image.flags.writeable = True   
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
        
     

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            # face = results.face_landmarks.landmark
            # face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('Foot.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        
     
        
cv2.destroyAllWindows()

# 3. Train Custom Model Using Scikit Learn

## 3.1 Read in Collected Data and Process

In [504]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [505]:
df = pd.read_csv('Foot.csv')

In [506]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [507]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)


## 3.2 Train Machine Learning Classification Model

In [508]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [509]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [510]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [511]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [512]:
fit_models['rc'].predict(X_test)

array(['Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
      

## 3.3 Evaluate and Serialize Model 

In [513]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [514]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 0.9859719438877755


In [515]:
fit_models['rc'].predict(X_test)


array(['Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
       'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot', 'Foot',
      

In [516]:
# Evaluation Metrics
from sklearn.metrics import classification_report, confusion_matrix

# Create empty lists to store true labels and predicted labels
true_labels = []
pred_labels = []

# Loop through the test set and make predictions
for i in range(X_test.shape[0]):
    # Make prediction
    pred = model.predict(X_test.iloc[[i]])
    # Append true and predicted labels to lists
    true_labels.append(y_test.iloc[i])
    pred_labels.append(pred[0])

# Print confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(true_labels, pred_labels))
print()
print("Classification Report:")
print(classification_report(true_labels, pred_labels))


Confusion Matrix:
[[492   7]
 [  0   0]]

Classification Report:
              precision    recall  f1-score   support

        Foot       1.00      0.99      0.99       499
   Incorrect       0.00      0.00      0.00         0

    accuracy                           0.99       499
   macro avg       0.50      0.49      0.50       499
weighted avg       1.00      0.99      0.99       499



C:\Users\acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning:

In [495]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_test)

# Compute predicted probabilities for each class
y_pred_prob = model.predict_proba(X_test)

# Compute ROC curve and AUC for each class
n_classes = len(model.classes_)
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and AUC
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_pred_prob.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot ROC curve for each class
plt.figure()
lw = 2
colors = cycle(['blue', 'red', 'green', 'purple', 'orange', 'brown', 'pink', 'gray', 'olive', 'cyan'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

# Plot micro-average ROC curve
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
         ''.format(roc_auc["micro"]),
         color='deeppink', linestyle='--', linewidth=4)

# Plot chance line
plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()


IndexError: index 1 is out of bounds for axis 1 with size 1

In [517]:
with open('Foot.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# 4. Make Detections with Model

In [518]:
with open('Foot.pkl', 'rb') as f:
    model = pickle.load(f)

In [529]:
# 
correctColor = (0, 256, 0)
inCorrectColor = (0, 0, 256)

directory = "images/Foot_engagement"
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
   for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".JPG"):
            # Load the image
            image = cv2.imread(os.path.join(directory, filename))

            # Recolor Feed
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False        

            # Make Detections
            results = holistic.process(image)
            # print(results.face_landmarks)

            # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks

            # Recolor image back to BGR for rendering
            image.flags.writeable = True   
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 

        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
                       
            # Concate rows
            row = pose_row
    
            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0][1]
        
            if body_language_prob < 0.50 or body_language_class == "Incorrect" :

                color = inCorrectColor
              
            else:
                color = correctColor
                
            if color == inCorrectColor:

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                    mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2)
                                    )
                
            else:
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                    mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2)
                                    )
            

            

            cv2.rectangle(image, (0,0), (250, 60), (0, 250, 0), -1) #kone thiyana box eka
                
                # Display Class
            cv2.putText(image, 'CLASS', (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0], (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                        
                # Display Probability
            cv2.putText(image, 'PROB', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob,2)), (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q' or 'Q'):
            break

cv2.destroyAllWindows()

In [520]:
cap = cv2.VideoCapture(0)
correctColor = (0, 256, 0)
inCorrectColor = (0, 0, 256)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
       

        # # 4. Pose Detections
        # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
        #                          mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
        #                          mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        #                          )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
                       
            # Concate rows
            row = pose_row
    
            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0][1]
        
            if body_language_prob < 0.45 or body_language_class == "Incorrect" :

                color = inCorrectColor
              
            else:
                color = correctColor
                
            if color == inCorrectColor:

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                    mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2)
                                    )
                
            else:
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                    mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2)
                                    )
            

            

            cv2.rectangle(image, (0,0), (250, 60), (0, 250, 0), -1) #kone thiyana box eka
                
                # Display Class
            cv2.putText(image, 'CLASS', (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0], (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                        
                # Display Probability
            cv2.putText(image, 'PROB', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob,2)), (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q' or 'Q'):
            break

cap.release()
cv2.destroyAllWindows()